## Parquet to SensorThings Streamlike

In [1]:
import csv
import re
import math
import time
import random
import numpy as np
import sys
import json
import requests
import os

In [2]:
import Helper

In [9]:
Helper.Helper.Kafkas['dev']

'smartaqnet-dev.teco.edu'

In [3]:
# dirList: sorted list of dir names
dirList = os.listdir(Helper.Helper.DataHome + "outAllSortByTimeStampAndIDBigAll/")
dirList = [ x for x in dirList if "2016" in x ]
dirList = sorted(dirList, key= lambda myDir: int(myDir.split("=")[1]))



In [4]:
Helper.Helper.

['TimeStamp=20160101',
 'TimeStamp=20160102',
 'TimeStamp=20160103',
 'TimeStamp=20160104',
 'TimeStamp=20160105',
 'TimeStamp=20160106',
 'TimeStamp=20160107',
 'TimeStamp=20160108',
 'TimeStamp=20160109',
 'TimeStamp=20160110',
 'TimeStamp=20160111',
 'TimeStamp=20160112',
 'TimeStamp=20160113',
 'TimeStamp=20160114',
 'TimeStamp=20160115',
 'TimeStamp=20160116',
 'TimeStamp=20160117',
 'TimeStamp=20160118',
 'TimeStamp=20160119',
 'TimeStamp=20160120',
 'TimeStamp=20160121',
 'TimeStamp=20160122',
 'TimeStamp=20160123',
 'TimeStamp=20160124',
 'TimeStamp=20160125',
 'TimeStamp=20160126',
 'TimeStamp=20160127',
 'TimeStamp=20160128',
 'TimeStamp=20160129',
 'TimeStamp=20160130',
 'TimeStamp=20160131',
 'TimeStamp=20160201',
 'TimeStamp=20160202',
 'TimeStamp=20160203',
 'TimeStamp=20160204',
 'TimeStamp=20160205',
 'TimeStamp=20160206',
 'TimeStamp=20160207',
 'TimeStamp=20160208',
 'TimeStamp=20160209',
 'TimeStamp=20160210',
 'TimeStamp=20160211',
 'TimeStamp=20160212',
 'TimeStamp

In [3]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
#spark conf
conf = ( SparkConf()
         .setMaster("local[*]")
         .setAppName('pyspark')
        )
ss = SparkSession.builder.config(conf=conf).getOrCreate()
sc = ss.sparkContext


In [4]:
#Common things
mytypes = ["Air-Temperature","Air-Humidity","Air-Pressure"]
urlHome = 'http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0'
#urlHome = 'http://smartaqnet.teco.edu:8080/FROST-Server/v1.0'
urlThings = urlHome + '/Things'
urlSensors = urlHome + '/Sensors'
urlObservedProperty = urlHome + '/ObservedProperties'
urlDataStream = urlHome + '/Datastreams'
urlFoI = urlHome + '/FeaturesOfInterest'

In [5]:
myPrefix = ""

In [6]:
#Observed Property
for mytype in mytypes:
    data ={
        "name": "Air-Temperature",
        "description": "The degree or intensity of heat present in the area",
        "definition": "http://www.qudt.org/qudt/owl/1.0.0/quantity/Instances.html#AreaTemperature",
        "@iot.id": myPrefix + "/ObservedProperty/" + mytype
    }
    p = requests.post(urlObservedProperty, json.dumps(data))
    if (p.status_code  == 201):
        print(201)
    else:
        print("Error:", p.status_code)

    for chunk in p.iter_content(chunk_size=128):
        print(chunk)

('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.
('Error:', 500)
Failed to store data.


In [8]:
from pyspark.sql import functions as F

from datetime import tzinfo, timedelta, datetime

def registerEntities(row):
    #Thing
    
    data =  {
        "name": "DWD-Sensor-" + row.ID,
        "description": "This is DWD-Sensor-" + row.ID,
        "@iot.id": myPrefix + row.ThUID,
        "Locations": [
        {
            "name": row.ThUID,
            "description": "This is the location of DWD-Sensor-" + row.ID,
            "encodingType": "application/vnd.geo+json",
            "location": {
              "type": "Point",
              "coordinates": [float(row.LatLon[0]), float(row.LatLon[1])]
            },
            "@iot.id": myPrefix+"Location/" + row.ThUID 
        }
      ]
        }
    p = requests.post(urlThings, json.dumps(data))
    if (p.status_code  == 201):
        print(201)
    else:
        print("Error:", p.status_code)

        for chunk in p.iter_content(chunk_size=128):
            print(chunk)

    #Sensor

    deviceAddr = "https://www.wunderground.com/personal-weather-station/dashboard?ID="+row.ID
    for mytype in mytypes:
        data = {
            "name": "DWD-Sensor-" + row.ID + "-" + "" + mytype,
            "description": "This is a Sensor from Netatmo Weather Station",
            "encodingType": "application/pdf",
            "metadata": deviceAddr,
            "@iot.id": myPrefix+row.ThUID +"-"+ mytype
        }
        p = requests.post(urlSensors, json.dumps(data))
        if (p.status_code  == 201):
            print(201)
        else:
            print("Error:", p.status_code)

        for chunk in p.iter_content(chunk_size=128):
            print(chunk)

    #DataStream

    for mytype in mytypes:
        data = {
          "name": mytype + "-DS",
          "description": "Datastream for recording temperature",
          "observationType": "http://www.opengis.net/def/observationType/OGC-OM/2.0/OM_Measurement",
          "@iot.id":myPrefix + row.ThUID + "/Datastreams/" + mytype + "-" + row.ID,
          "unitOfMeasurement": {
            "name": "Degree Celsius",
            "symbol": "degC",
            "definition": "http://www.qudt.org/qudt/owl/1.0.0/unit/Instances.html#DegreeCelsius"
          },
          "Thing":{"@iot.id":myPrefix+row.ThUID},
          "ObservedProperty":{"@iot.id":myPrefix+"/ObservedProperty/" + mytype},
          "Sensor":{"@iot.id":myPrefix+row.ThUID +"-"+ mytype}
        }
        p = requests.post(urlDataStream, json.dumps(data))
        if (p.status_code  == 201):
            print(201)
        else:
            print("Error:", p.status_code)

        for chunk in p.iter_content(chunk_size=128):
            print(chunk)

    # FoI
    data = {
      "name": "Weather Station-" + row.ID,
      "description": "Weather Station-" + row.ID,
      "encodingType": "application/vnd.geo+json",
        "@iot.id": myPrefix+row.ThUID + "/FoI/"+row.ID+"/"+row.LatLon[0]+":"+row.LatLon[1],
      "feature": {
        "type": "Point",
        "coordinates": [float(row.LatLon[0]), float(row.LatLon[1])]
      }
    }
    p = requests.post(urlFoI, json.dumps(data))
    if (p.status_code  == 201):
        print(201)
    else:
        print("Error:", p.status_code)

    for chunk in p.iter_content(chunk_size=128):
        print(chunk)




for myDir in dirList:
    inputDir = "./out/outAllSortByTimeStampAndIDBigAll/"+myDir
    dataFileDF = ss.read.option("basepath",inputDir).parquet(inputDir)#+"TimeStamp=20160504/ID=I72406BI1")
    dataFileDF = dataFileDF.withColumnRenamed("AitTemperature","AirTemperature")
    dataFileDF = dataFileDF.filter("(not isnull(ID)) and ID !=''")
    idDF = dataFileDF["ID","Latitude","Longitude"]
    
    urlInst = "edu.teco.wang"
    idDF = idDF.withColumn("LatLon",F.array("Latitude","Longitude"))
    idDF = idDF.groupBy(idDF.ID).agg(F.first(idDF.LatLon).alias("LatLon"))

    idDF = idDF.withColumn("ThUID",F.format_string("%s/%s",F.lit(urlInst),idDF.ID))
    idDF = idDF.withColumn("DsUID",F.format_string("%s/Datastreams/%s",F.lit(urlInst),idDF.ID))
    print(idDF.count())
    for row in idDF.collect():
        '''
        #Thing

        data =  {
            "name": "DWD-Sensor-" + row.ID,
            "description": "This is DWD-Sensor-" + row.ID,
            "@iot.id": myPrefix + row.ThUID,
            "Locations": [
            {
                "name": row.ThUID,
                "description": "This is the location of DWD-Sensor-" + row.ID,
                "encodingType": "application/vnd.geo+json",
                "location": {
                  "type": "Point",
                  "coordinates": [float(row.LatLon[0]), float(row.LatLon[1])]
                },
                "@iot.id": myPrefix+"Location/" + row.ThUID 
            }
          ]
            }
        p = requests.post(urlThings, json.dumps(data))
        if (p.status_code  == 201):
            print(201)
        else:
            print("Error:", p.status_code)

            for chunk in p.iter_content(chunk_size=128):
                print(chunk)

        #Sensor

        deviceAddr = "https://www.wunderground.com/personal-weather-station/dashboard?ID="+row.ID
        for mytype in mytypes:
            data = {
                "name": "DWD-Sensor-" + row.ID + "-" + "" + mytype,
                "description": "This is a Sensor from Netatmo Weather Station",
                "encodingType": "application/pdf",
                "metadata": deviceAddr,
                "@iot.id": myPrefix+row.ThUID +"-"+ mytype
            }
            p = requests.post(urlSensors, json.dumps(data))
            if (p.status_code  == 201):
                print(201)
            else:
                print("Error:", p.status_code)

            for chunk in p.iter_content(chunk_size=128):
                print(chunk)

        #DataStream

        for mytype in mytypes:
            data = {
              "name": mytype + "-DS",
              "description": "Datastream for recording temperature",
              "observationType": "http://www.opengis.net/def/observationType/OGC-OM/2.0/OM_Measurement",
              "@iot.id":myPrefix + row.ThUID + "/Datastreams/" + mytype + "-" + row.ID,
              "unitOfMeasurement": {
                "name": "Degree Celsius",
                "symbol": "degC",
                "definition": "http://www.qudt.org/qudt/owl/1.0.0/unit/Instances.html#DegreeCelsius"
              },
              "Thing":{"@iot.id":myPrefix+row.ThUID},
              "ObservedProperty":{"@iot.id":myPrefix+"/ObservedProperty/" + mytype},
              "Sensor":{"@iot.id":myPrefix+row.ThUID +"-"+ mytype}
            }
            p = requests.post(urlDataStream, json.dumps(data))
            if (p.status_code  == 201):
                print(201)
            else:
                print("Error:", p.status_code)

            for chunk in p.iter_content(chunk_size=128):
                print(chunk)

        # FoI
        data = {
          "name": "Weather Station-" + row.ID,
          "description": "Weather Station-" + row.ID,
          "encodingType": "application/vnd.geo+json",
            "@iot.id": myPrefix+row.ThUID + "/FoI/"+row.ID+"/"+row.LatLon[0]+":"+row.LatLon[1],
          "feature": {
            "type": "Point",
            "coordinates": [float(row.LatLon[0]), float(row.LatLon[1])]
          }
        }
        p = requests.post(urlFoI, json.dumps(data))
        if (p.status_code  == 201):
            print(201)
        else:
            print("Error:", p.status_code)

        for chunk in p.iter_content(chunk_size=128):
            print(chunk)
        '''
    for hourInt in range(0,24):
        oneHourDataFileDF = dataFileDF.rdd.filter(lambda row: int(row.Time.split(':')[0]) == hourInt)
        print(oneHourDataFileDF.count())
        for row in oneHourDataFileDF.collect():
            

            class TZ(tzinfo):
                def utcoffset(self, dt): return timedelta(minutes=120)
            isoTime =datetime(int(row.Date.split('-')[0]),
                         int(row.Date.split("-")[1]),
                         int(row.Date.split("-")[2]),
                         int(row.Time.split(':')[0]),
                         int(row.Time.split(':')[1]),
                         int(row.Time.split(':')[2]),tzinfo=TZ()).isoformat()




            urlObs = urlHome+"/Datastreams('"+myPrefix+"edu.teco.wang/"+row.ID+"/Datastreams/" + "Air-Temperature" + "-" + row.ID+"')/Observations"
            data = {
              "phenomenonTime": isoTime,
              "resultTime" : isoTime,
              "result" : row.AirTemperature,
              "FeatureOfInterest": {
                  "@iot.id": myPrefix + urlInst+"/"+row.ID + "/FoI/"+row.ID+"/"+row.Latitude+":"+row.Longitude
              }
            }
            p = requests.post(urlObs, json.dumps(data))

            if (p.status_code  == 201):
                print(p.headers["location"])
            else:
                print("Error:", p.status_code)
                #print(row.ID)
                #print(idDF.rdd.take(5))
                registerEntities(idDF.rdd.filter(lambda aRow:aRow.ID == row.ID).first())

            for chunk in p.iter_content(chunk_size=128):
                print(chunk)
    time.sleep(10)
    

9978
14761
http://localhost:8080/FROST-Server/v1.0/Observations('25f1f50e-641e-11e8-8042-f78db5110699')
http://localhost:8080/FROST-Server/v1.0/Observations('25f745cc-641e-11e8-8042-376ae93e1b41')
http://localhost:8080/FROST-Server/v1.0/Observations('25f9ae34-641e-11e8-8042-038fe986c4d4')
http://localhost:8080/FROST-Server/v1.0/Observations('25fc3050-641e-11e8-8042-2360180cec8a')
http://localhost:8080/FROST-Server/v1.0/Observations('25fec4be-641e-11e8-8042-17eb5455536d')
http://localhost:8080/FROST-Server/v1.0/Observations('26014f72-641e-11e8-8042-af43a1de7eca')
http://localhost:8080/FROST-Server/v1.0/Observations('2603edb8-641e-11e8-8042-338d5bc03d10')
http://localhost:8080/FROST-Server/v1.0/Observations('2607a462-641e-11e8-8042-ef2602c84176')
http://localhost:8080/FROST-Server/v1.0/Observations('260a104e-641e-11e8-8042-67f6e303b93e')
http://localhost:8080/FROST-Server/v1.0/Observations('260ccb7c-641e-11e8-8042-5bfcb8f31bbd')
http://localhost:8080/FROST-Server/v1.0/Observations('260f2

http://localhost:8080/FROST-Server/v1.0/Observations('2739043e-641e-11e8-8042-6b1b9e3607aa')
http://localhost:8080/FROST-Server/v1.0/Observations('273ba900-641e-11e8-8042-73f4bea593a5')
http://localhost:8080/FROST-Server/v1.0/Observations('273e41a6-641e-11e8-8042-73b8f53979c7')
http://localhost:8080/FROST-Server/v1.0/Observations('2740d20e-641e-11e8-8042-73095b4f0a8f')
http://localhost:8080/FROST-Server/v1.0/Observations('27432d56-641e-11e8-8042-8b44ea00e283')
http://localhost:8080/FROST-Server/v1.0/Observations('27459bcc-641e-11e8-8042-470808d5c7bd')
http://localhost:8080/FROST-Server/v1.0/Observations('274a8948-641e-11e8-8042-0b17bcd4bca9')
http://localhost:8080/FROST-Server/v1.0/Observations('274d60f0-641e-11e8-8042-0b59a6c1d01c')
http://localhost:8080/FROST-Server/v1.0/Observations('274fffa4-641e-11e8-8042-9f7b23f04b0c')
http://localhost:8080/FROST-Server/v1.0/Observations('27542bb0-641e-11e8-8042-ef6a87d32e44')
http://localhost:8080/FROST-Server/v1.0/Observations('27577ce8-641e-11

http://localhost:8080/FROST-Server/v1.0/Observations('28563f94-641e-11e8-8042-0319f913421a')
http://localhost:8080/FROST-Server/v1.0/Observations('285b3486-641e-11e8-8042-43ba331a28bf')
http://localhost:8080/FROST-Server/v1.0/Observations('28602608-641e-11e8-8042-cb179be8294a')
http://localhost:8080/FROST-Server/v1.0/Observations('28631502-641e-11e8-8042-373e716be4da')
http://localhost:8080/FROST-Server/v1.0/Observations('286573a6-641e-11e8-8042-e335d1e156af')
http://localhost:8080/FROST-Server/v1.0/Observations('28680aee-641e-11e8-8042-fbdf416fc9a7')
http://localhost:8080/FROST-Server/v1.0/Observations('286a6d7a-641e-11e8-8042-9f7613ca7041')
http://localhost:8080/FROST-Server/v1.0/Observations('286d14bc-641e-11e8-8042-973932f9e1bc')
http://localhost:8080/FROST-Server/v1.0/Observations('28712d4a-641e-11e8-8042-d313b3cc5b9b')
http://localhost:8080/FROST-Server/v1.0/Observations('2874ce5a-641e-11e8-8042-3f3818258759')
http://localhost:8080/FROST-Server/v1.0/Observations('28773f50-641e-11

http://localhost:8080/FROST-Server/v1.0/Observations('2974a88e-641e-11e8-8042-f3918e815ecf')
http://localhost:8080/FROST-Server/v1.0/Observations('29792544-641e-11e8-8042-7fa006906e42')
http://localhost:8080/FROST-Server/v1.0/Observations('297d3fb2-641e-11e8-8042-df0903d0e0f4')
http://localhost:8080/FROST-Server/v1.0/Observations('29807754-641e-11e8-8042-b79c94d61601')
http://localhost:8080/FROST-Server/v1.0/Observations('29844cbc-641e-11e8-8042-9363db01ed90')
http://localhost:8080/FROST-Server/v1.0/Observations('29866b96-641e-11e8-8042-ff095cfcbdba')
http://localhost:8080/FROST-Server/v1.0/Observations('2988fdd4-641e-11e8-8042-d7c51ec8aacf')
http://localhost:8080/FROST-Server/v1.0/Observations('298bb344-641e-11e8-8042-734df4ae3e8c')
http://localhost:8080/FROST-Server/v1.0/Observations('298e6468-641e-11e8-8042-b7ac640f367e')
http://localhost:8080/FROST-Server/v1.0/Observations('299095ee-641e-11e8-8042-c37bebf8aac2')
http://localhost:8080/FROST-Server/v1.0/Observations('29934ce4-641e-11

http://localhost:8080/FROST-Server/v1.0/Observations('2a9636ba-641e-11e8-8042-97fe84724529')
http://localhost:8080/FROST-Server/v1.0/Observations('2a99800e-641e-11e8-8042-bf271dc2674c')
http://localhost:8080/FROST-Server/v1.0/Observations('2a9f8af8-641e-11e8-8042-339d40acfb12')
http://localhost:8080/FROST-Server/v1.0/Observations('2aa465c8-641e-11e8-8042-a328e7e821d6')
http://localhost:8080/FROST-Server/v1.0/Observations('2aa8c816-641e-11e8-8042-4310309e4472')
http://localhost:8080/FROST-Server/v1.0/Observations('2aac0c92-641e-11e8-8042-ffed6f52e28a')
http://localhost:8080/FROST-Server/v1.0/Observations('2aaedc92-641e-11e8-8042-d7bd4e2bc590')
http://localhost:8080/FROST-Server/v1.0/Observations('2ab1a26a-641e-11e8-8042-03757dbc7dea')
http://localhost:8080/FROST-Server/v1.0/Observations('2ab4e6fa-641e-11e8-8042-ef182ecc6c0a')
http://localhost:8080/FROST-Server/v1.0/Observations('2ab82eaa-641e-11e8-8042-bfec03a50843')
http://localhost:8080/FROST-Server/v1.0/Observations('2abb47ac-641e-11

...

201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/INORDRHE460/Datastreams/Air-Temperature-INORDRHE460
http://localhost:8080/FROST-Server/v1.0/Observations('e07654a6-641e-11e8-8042-1745cd1b147f')
('Error:', 400)
201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/IGREVENB2/Datastreams/Air-Temperature-IGREVENB2
http://localhost:8080/FROST-Server/v1.0/Observations('e1dc04d0-641e-11e8-8042-177c56aaf93c')
('Error:', 400)
201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/INORDRHE525/Datastreams/Air-Temperature-INORDRHE525
http://localhost:8080/FROST-Server/v1.0/Observations('e2e42254-641e-11e8-8042-8b7b95002b94')
('Error:', 400)
201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/IBOCHUM273/Datastreams/Air-Temperature-IBOCHUM273
('Error:', 400)
201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/IERFURT27/Datastreams/Air-Temperature-IERFURT

201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/IRGLING2/Datastreams/Air-Temperature-IRGLING2
('Error:', 400)
201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/IERFTSTA14/Datastreams/Air-Temperature-IERFTSTA14
('Error:', 400)
201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/IMENDEN2/Datastreams/Air-Temperature-IMENDEN2
('Error:', 400)
201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/IDORMAGE11/Datastreams/Air-Temperature-IDORMAGE11
('Error:', 400)
201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/ILIMBACH3/Datastreams/Air-Temperature-ILIMBACH3
('Error:', 400)
201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/IBADENWR117/Datastreams/Air-Temperature-IBADENWR117
http://localhost:8080/FROST-Server/v1.0/Observations('0a89fea0-641f-11e8-8042-c7ef8be8fd99')
('Error:', 400)
201
201
201
201
201
201
201

('Error:', 400)
201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/ISACHSEN265/Datastreams/Air-Temperature-ISACHSEN265
http://localhost:8080/FROST-Server/v1.0/Observations('29aef600-641f-11e8-8042-c33cda42e2e7')
('Error:', 400)
201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/INORDRHE319/Datastreams/Air-Temperature-INORDRHE319
http://localhost:8080/FROST-Server/v1.0/Observations('2a82aa90-641f-11e8-8042-6be95744a454')
('Error:', 400)
201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/IZRBIG2/Datastreams/Air-Temperature-IZRBIG2
http://localhost:8080/FROST-Server/v1.0/Observations('2bb8b38c-641f-11e8-8042-6bfbb471eabc')
('Error:', 400)
201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/INEUSS84/Datastreams/Air-Temperature-INEUSS84
('Error:', 400)
201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/IOSTRACH3/Datastreams/Air-Temperature

201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/IMOERS60/Datastreams/Air-Temperature-IMOERS60
http://localhost:8080/FROST-Server/v1.0/Observations('509d6d82-641f-11e8-8042-ff828e1665d3')
('Error:', 400)
201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/IGROEFEH6/Datastreams/Air-Temperature-IGROEFEH6
http://localhost:8080/FROST-Server/v1.0/Observations('51ddfa72-641f-11e8-8042-73b22f56f906')
('Error:', 400)
201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/IBADENWR481/Datastreams/Air-Temperature-IBADENWR481
http://localhost:8080/FROST-Server/v1.0/Observations('5334fbb4-641f-11e8-8042-c3fc024f56fb')
('Error:', 400)
201
201
201
201
201
201
201
201
No such entity 'Datastream' with id edu.teco.wang/IWEILERS5/Datastreams/Air-Temperature-IWEILERS5
http://localhost:8080/FROST-Server/v1.0/Observations('542bc46c-641f-11e8-8042-63e6164e34a1')
('Error:', 400)
201
201
201
201
201
201
201
201
No such en

KeyboardInterrupt: 

In [ ]:
urlObs = urlHome+"/Datastreams('"+myPrefix+"edu.teco.wang/"+row.ID+"/Datastreams/" + "Air-Temperature" + "-" + row.ID+"')/Observations"

p = requests.post(urlObs, json.dumps(
{
  "phenomenonTime": isoTime,
  "resultTime" : isoTime,
  "result" : row.AirTemperature,
  "FeatureOfInterest": {
      "@iot.id": myPrefix + urlInst+"/"+row.ID + "/FoI/"+row.ID+"/"+row.Latitude+":"+row.Longitude
  }
}
))

if (p.status_code  == 201):
    print(p.headers["location"])
else:
    print("Error:", p.status_code)

for chunk in p.iter_content(chunk_size=128):
    print(chunk)

In [ ]:

#idDF = idDF.withColumn("FoIUID",F.format_string("%s/FeaturesOfInterest/%s",idDF.ThID,idDF.ID))
#idDF = idDF.withColumn("SensorUID",F.format_string("%s/Sensors/%s",idDF.ThID,idDF.ID))


In [ ]:
row = idDF.collect()[0]

row

In [ ]:
# Observations
dataFileDF.show(10)

In [ ]:
row = dataFileDF.collect()[0]
row

In [ ]:
from datetime import tzinfo, timedelta, datetime
for row in dataFileDF.collect():
    

    class TZ(tzinfo):
        def utcoffset(self, dt): return timedelta(minutes=120)
    isoTime =datetime(int(row.Date.split('-')[0]),
                 int(row.Date.split("-")[1]),
                 int(row.Date.split("-")[2]),
                 int(row.Time.split(':')[0]),
                 int(row.Time.split(':')[1]),
                 int(row.Time.split(':')[2]),tzinfo=TZ()).isoformat()
    
    
    
    
    urlObs = urlHome+"/Datastreams('edu.teco.wang/"+row.ID+"/Datastreams/" + "Air-Temperature" + "-" + row.ID+"')/Observations"
    data ={
      "phenomenonTime": isoTime,
      "resultTime" : isoTime,
      "result" : row.AirTemperature
    }
    p = requests.post(urlObs, json.dumps(data))
    if (p.status_code  == 201):
        print(p.headers["location"])
    else:
        print("Error:", p.status_code)

    for chunk in p.iter_content(chunk_size=128):
        print(chunk)


In [ ]:
urlObs

In [ ]:
p.headers["location"]

In [ ]:
from datetime import tzinfo, timedelta, datetime
row = dataFileDF.first()
row.Date.split("-")[0]
class TZ(tzinfo):
    def utcoffset(self, dt): return timedelta(minutes=120)
datetime(int(row.Date.split('-')[0]),
         int(row.Date.split("-")[1]),
         int(row.Date.split("-")[2]),
         int(row.Time.split(':')[0]),
         int(row.Time.split(':')[1]),
         int(row.Time.split(':')[2]),tzinfo=TZ()).isoformat()

## Tryouts

In [ ]:
row = idDF.collect()[1]
urlHome = 'http://smartaqnet.teco.edu:8080/FROST-Server/v1.0'
urlThings = urlHome + '/Things'
data =  {
        "name": "DWD-Sensor-noSens" + row[0],
        "description": "DWD_Sensor-" + row[0],
        "@iot.id": "DWD-Sensor/noSensblblb" + row[0]
    }
p = requests.post(urlThings, json.dumps(data))
if (p.status_code  == 201):
    print(201)
else:
    print("Error:", p.status_code)
    
    for chunk in p.iter_content(chunk_size=128):
        print(chunk)

In [ ]:

idDF = idDF.groupBy(idDF.ID).agg(F.first(idDF.LatLon).alias("LatLon"))#.show(3)#.agg(F.first(idDF.Longitude))

In [ ]:
urlHome = 'http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0'
#urlHome = 'http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0'
urlThings = urlHome + '/Things'

In [ ]:

#idDF2 = idDF.select("ID","LatLon").rdd.map(DfToSensorthings)

In [ ]:
row = idDF.collect()[1]
import requests
urlHome = 'http://smartaqnet.teco.edu:8080/FROST-Server/v1.0'
urlThings = urlHome + '/Things'
sensorAddr = "https://www.wunderground.com/personal-weather-station/dashboard?ID="+row[0]
data =  {
        "name": "DWD-Sensor-" + row[0],
        "description": "DWD_Sensor-" + row[0],
        "@iot.id": "DWD-Sensor/" + row[0]
    }
p = requests.get(urlThings+"('DWD-Sensor/noSensblbl" + row[0]+"')")
if (p.status_code  == 200):
    print(200)
else:
    print("Error:", p.status_code)
    
for chunk in p.iter_content(chunk_size=128):
    print(chunk)

In [ ]:
row

In [ ]:
p = requests.post(urlThings, json.dumps(data))
if (p.status_code  == 201):
    print("Creation successful")
else:
    print("Error:", p.status_code)
    for chunk in p.iter_content(chunk_size=128):
        print(chunk)
        

In [ ]:
#Delete a thing

deleteID = urlThings + "('DWD-Sensors/IBREUNA2')"
print(deleteID)
p = requests.delete(urlThings)
if (p.status_code  == 201):
    print("Deletion successful")
else:
    print("Error:", p.status_code)
    for chunk in p.iter_content(chunk_size=128):
        print(chunk)

In [ ]:
#Query a thing

getID = urlThings + "(\"teco.edu/Test-2\")"
print(getID)
p = requests.get(urlThings)
if (p.status_code  == 201):
    print("Get successful")
else:
    print("Error:", p.status_code)
    for chunk in p.iter_content(chunk_size=128):
        print(chunk)

In [ ]:
p.content

In [ ]:
sensors = idDF.collect()
for sensor in sensors:
    print(DfToSensorthingsPost(sensor))
    #self link

In [ ]:
import requests

In [ ]:
dataFileDF = ss.read.parquet(inputDir)
outDir = "./outDir/outAllAll5/"
print(csv.__file__)

In [ ]:
validFileNames = [inputDir+f for f in os.listdir(inputDir) if ('.' not in f) and ("part-" in f)]
validFileNames

In [ ]:
for aFile in validFileNames:
    with open(aFile) as csvfile:
        for row in csvfile:
            k = row[1:9]
            v = row[13:-3]+'\n'
            vNew = re.sub(';','\n',v)
            outFile = open(outDir+k,"w")
            outFile.write(vNew)
            outFile.close()


In [ ]:
row = csvfile.readline()

In [ ]:
k = row[1:9]
v = row[13:-3].split(";")
k,v

In [ ]:
with open(inputFile) as csvfile:
    for row in csvfile:
        k = row[1:9]
        v = row[13:-3]+'\n'
        vNew = re.sub(';','\n',v)
        outFile = open(outDir+k,"w")
        outFile.write(vNew)
        outFile.close()

In [ ]:
import re

print(vNew)

In [ ]:
outFile = open(outDir+k,"w")
outFile.write(vNew)
outFile.close()

In [ ]:
with open(inputFile,'rb') as csvfile:
    reader = csv.Reader(csvfile)
    row = reader.__next__()
row

In [ ]:
row

In [ ]:
all=string.maketrans('','')
nodigs=all.translate(all, string.digits)

In [ ]:
class Del:
  def __init__(self, keep=string.digits):
    self.comp = dict((ord(c),c) for c in keep)
  def __getitem__(self, k):
    return self.comp.get(k)

In [ ]:
with open(inputPath) as csvfile:
    reader = csv.DictReader(csvfile)
    for i in range(10):
        row = reader.__next__()
        
        lineID = row['id']
        timeStamp = row['time'].translate(Del())
        print(lineID, timeStamp)
        